### Задание 1.
##### Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

##### Дополнительная часть (необязательная)¶
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
import pandas as pd

In [4]:
from bs4 import SoupStrainer

In [5]:
import time

In [6]:
from datetime import datetime

In [7]:
KEYWORDS = ['Android', 'Siemens', 'Linux']

In [ ]:
#keywords_strain = SoupStrainer("GitHub")

In [8]:
DESIRED_HUBS = ['python', 'bigdata', 'java']

In [9]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [ ]:
#req.text
#soup

In [10]:
posts = soup.find_all('article', class_='post post_preview')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/Audioman/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/7fd/66e/5a3/7fd66e5a3b645f7ecdd9c99e5893c15d.png" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">Audioman</span>
 </a>
 <span class="post__time">сегодня в 23:17</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/audiomania/blog/536656/">Лайвстрим в рамках закона: как соцсети ограничивают использование музыки, к чему это приводит и что с этим делать</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/company/audiomania/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'fee

In [12]:
#вариант 1
#получается найти все данные, но вот эту строку "if any([hub_lower in desired for desired in DESIRED_HUBS]):" приведенную в
#ноутбуке так и не смог применить. Задачу получается решить через сортировку в Пандасе .str.contains

habr_news = pd.DataFrame()
for post in posts:
    time.sleep(0.3)
    
    
                    
    #print('post', post_id)
    date = post.find('span', class_='post__time').text #.replace('сегодня', datetime.now())
    
    #date = pd.to_datetime(post.find('span', class_='post__time').get('datetime'), dayfirst=True).date()
    #print(date)
    title = post.find('h2', class_='post__title').text.strip()
    #print(title)
    #link = post.find('a', class_='post__title_link').text
    link = post.find('a', class_='post__title_link').get('href')
    #print(link.attrs.get('href'))
    text = post.find('div', class_='post__body post__body_crop').text.strip()
    #print(text)
    #hubs = post.find_all('a', class_='article')
    title_plus_text = title + text #).lower()
        
            #hub_lower = hub.text.lower()
           # ищем вхождение хотя бы одного желаемого хаба
    #if any([title_plus_text in desired for desired in KEYWORDS]):
    for i in KEYWORDS:
        if i in title_plus_text:
            row = {'date': date, 'title': title, 'link': link, 'text': text}
            habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
#habr_news
#habr_sort = habr_news[habr_news['title'].str.contains('Android|лидар') | habr_news['text'].str.contains('python|Android')]
#habr_sort
habr_news

,date,title,link,text
0,сегодня в 21:01,"Чаты на вебсокетах, когда на бэкенде WAMP. Теп...",https://habr.com/ru/company/funcorp/blog/537144/,Мой коллега уже писал про наш опыт разработки ...
0,сегодня в 19:28,Промышленный интернет вещей в ПЛК Simatic S7-1...,https://habr.com/ru/post/537232/,Обнаружил в базе знаний Siemens (SIOS) интерес...
0,сегодня в 16:57,Linux exploits,https://habr.com/ru/company/otus/blog/537192/,Операционная система Linux доказала миру всю с...


In [ ]:
for post in posts:
    today = str(datetime.today())
    date = post.find('span', class_='post__time').text.replace('сегодня', today)
    
    title = post.find('h2', class_='post__title').text
    
    link = post.find('a', class_='post__title_link').get('href')
    
    text = post.find('div', class_='post__body post__body_crop').text.strip()
    
    title_plus_text = title + text #).lower()
    
    for i in KEYWORDS:
        if i in text:
            print(type(date), date)
            print(text)